In [1]:
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
import pandas as pd

**Collecting Mars news**

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://mars.nasa.gov/news/'

In [5]:
browser.visit(url)

In [6]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [7]:
title = soup.find('ul', class_='item_list ').find('li', class_='slide').find('div', class_='content_title')\
.find('a').text

In [8]:
paragraph = soup.find('ul', class_='item_list ').find('li', class_='slide').\
find('div', class_='article_teaser_body').text

In [9]:
print(title)
print(paragraph)

A Pale Blue Dot, As Seen by a CubeSat
One of NASA's MarCO CubeSats has taken its first image.


**Getting Mars image link**

In [10]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [19]:
browser.visit(url)

**Open full image**

In [20]:
browser.find_by_css('div[class="default floating_text_area ms-layer"]').find_by_css('footer')\
.find_by_css('a[class="button fancybox"]').click()

In [32]:
browser.find_by_css('div[id="fancybox-lock"]').find_by_css('div[class="buttons"]')\
.find_by_css('a[class="button"]').click()

In [40]:
browser.find_by_css('div[id="page"]').find_by_css('section[class="content_page module"]')\
.find_by_css('figure[class="lede"]').find_by_css('a')['href']

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18297_hires.jpg'

In [ ]:
https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16153_hires.jpg